In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.3 MB/s eta 0:00:00


In [1]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

# initializing the vertex ai

In [3]:

PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east1"
vertexai.init(project=PROJECT_ID, location=REGION)

# load the generative model

In [5]:

model = GenerativeModel("gemini-pro")

# Within the context of using a generative AI model, "ideation" means using the model to help you brainstorm new content: article titles, sections to include in a document, interview questions, etc.

# This is a case where you may want the model to be more creative or surprising in the content it generates. To enable that, create a GenerationConfig object by copying the code below to a new code cell and running it:



In [6]:

creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)



In [7]:
prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

## Strategies for Overcoming Writer's Block

Writer's block can be a frustrating and discouraging experience, but don't worry, there are many strategies you can use to overcome it. Here are a few ideas:

**Change your environment:** 

* **Move to a different location:** Leave your desk and find a new spot to write, like a coffee shop, library, or park. Sometimes a change of scenery can spark new ideas.
* **Work in a different format:** If you're usually writing on your computer, try writing on paper or using a different writing tool. This can help you break out of your usual patterns and get your creative juices flowing.
* **Listen to music or ambient sounds:** Put on some calming music or nature sounds to help you relax and focus. 

**Free writing:**

* **Set a timer for 5 minutes and write whatever comes to mind, without stopping or editing.** This can help you get your thoughts flowing and loosen up your writing muscles.
* **Do a writing sprint:** Set a goal for a certain number of 

In [8]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

## Ten Interview Questions for a Prompt Engineering Role:

1. **Explain the prompt engineering process in your own words.**
2. **Describe a project where you designed and implemented a successful prompt for a large language model. What challenges did you encounter and how did you overcome them?**
3. **What are the different types of prompts and when would you use each type?**
4. **How do you evaluate the effectiveness of a prompt? What metrics do you use?**
5. **What are some of the ethical considerations when designing prompts for large language models?**
6. **How do you stay up-to-date on the latest research and advancements in prompt engineering?**
7. **Can you describe a situation where you had to troubleshoot a prompt that was not performing as expected? What steps did you take to diagnose and fix the problem?**
8. **How do you collaborate with other engineers and data scientists to ensure that your prompts are aligned with the overall goals of the project?**
9. **What are your th

## Task 4. Text Classification
In modern enterprises, a lot of work is dedicated to putting the right information in the right place. Routing customer support emails to the correct team or sending customer reviews to the relevant department are two examples. To do this, you could ask generative AI to classify those blocks of text to determine the right team.

## Classification can have many possible applications, including:

Sentiment analysis
Topic classification
Spam detection
Intent recognition
Language identification
Toxicity detection
Emotion detection


In [10]:
# For a more tightly-constrained summary, you may want to decrease the temperature and top_p parameters:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [11]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

## Intent: Making a reservation

The user's intent is to make a reservation. They are specifically requesting a table for two at a restaurant called Juan for May 1st. 



In [12]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment 



In [13]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [14]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,"""positive"""
1,all i can say is that you will be happy after ...,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


Now you can call the classification_report function from sklearn to return classification metrics based on comparing the sentiment_groundtruth and predicted sentiment_prediction fields:

In [16]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

  "positive"       0.00      0.00      0.00         0
    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.75      0.62      0.67         4
weighted avg       1.00      0.75      0.83         4


# Task 5. Text Summarization

In [17]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [19]:
prompt = """
Provide a very short summary, no more than three sentences in line by line, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Quantum computers use qubits, which are sensitive to errors.
Quantum error correction protects information by encoding it across multiple qubits.
This allows for large-scale quantum computers with low error rates for useful calculations.


In [20]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary in four bullet points:

* **Quantum computers use qubits, which are sensitive to errors.** Even small disturbances can cause calculation errors, especially as quantum computers grow in size.
* **Current error rates are too high for useful applications.** The best quantum algorithms require much lower error rates than we have today.
* **Quantum error correction is the solution.** It encodes information across multiple qubits to create a "logical qubit" with lower error rates.
* **Logical qubits will enable useful quantum algorithms.** By encoding physical qubits into logical qubits, we can achieve the low error rates needed for practical applications. 


In [21]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary of Conversation:

* **Customer:** Larry received the wrong item.
* **Desired Resolution:** Return the item and receive a refund.
* **Resolution:** Support Agent processed the refund.
* **Refund Timeline:** Customer will receive the refund within 14 days.

## To-Do's for Support Agent:

* **None.** The Support Agent has already completed the necessary actions. 


# Task 6. Text Extraction
Text Extraction is the process of pulling structured fields from unstructured text. By unstructured text, we mean text that lacks a computer-readable structure like CSV, JSON, or YAML, even if a human can identify some structure (like in the ingredient list of a recipe).

## Some specific types of extraction include:

Named entity recognition (NER): Extract named entities from text, including people, places, organizations, and dates.

Relation extraction: Extract the relationships between entities in text, such as family relationships between people.

Event extraction: Extract events from text, such as project milestones and product launches.

Question answering: Extract information from text to answer a question.


In [22]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```


# Task 7. Question Answering
Answering questions is one of the most common and most impressive uses of generative AI. The information the model returns could come from patterns in data the model was trained on or from additional information you provide the model as contex

In [23]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

Dwight D. Eisenhower was President of the United States in 1955. He belonged to the Republican Party.


In [24]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R